In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:0000:010:01


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 5.8 MB/s eta 0:00:0000:01


In [ ]:
import json
import tqdm
import pandas as pd
train = json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/train.json"))

In [ ]:
type(train)

list

In [ ]:
train_df = pd.DataFrame(train)

In [ ]:
def process_documents_to_dict_with_sets(documents):
    all_entities_dicts = []

    for document in documents:
        tokens = document["tokens"]
        labels = document["labels"]
        entities_dict = {}
        current_entity = ""
        current_label = ""

        for token, label in zip(tokens, labels):
            if label.startswith("B-"):
                if current_entity:  # Save the previous entity if it exists
                    entities_dict.setdefault(current_label, set()).add(current_entity)
                current_entity = token  # Start a new entity
                current_label = label[2:]  # Extract entity type without the B-
            elif label.startswith("I-") and label[2:] == current_label:
                current_entity += " " + token  # Continue building the entity
            else:  # Either an 'O' label or a different entity type
                if current_entity:  # Save the previous entity if it exists
                    entities_dict.setdefault(current_label, set()).add(current_entity)
                    current_entity = ""
                    current_label = ""

        # Check if there's an entity still being built
        if current_entity:
            entities_dict.setdefault(current_label, set()).add(current_entity)

        all_entities_dicts.append((entities_dict, set([l.split("-")[-1] for l in labels]), document["full_text"]))

    return all_entities_dicts

In [ ]:
prompt_text = "Rephrase the following essay in different way while substituting following PIIs with the given ones,\n {}\n\n-------------------------------------\n{}"

In [ ]:
from faker import Faker
import random
import string


fake = Faker()
common_email_domains = [
    "gmail.com",
    "yahoo.com",
    "outlook.com",
    "icloud.com",
    "aol.com",
    "mail.com",
    "yandex.com",
    "protonmail.com",
    "zoho.com"
]

def generate_random_id():
    patterns = [
        lambda: str(random.randint(100000000000, 999999999999)),  # 12-digit numbers
        lambda: f"{''.join([random.choice(string.ascii_uppercase+string.ascii_lowercase) for i in range(2)]).title()}:{random.randint(100000000000, 999999999999)}",  # Prefix:numbers
        lambda: ''.join(random.choices(string.ascii_uppercase + string.digits, k=8)),  # Alphanumeric (8 characters)
        lambda: f"{''.join(random.choices(string.ascii_uppercase +string.ascii_lowercase + string.digits, k=8))}_{random.randint(1000000000, 9999999999)}",  # Unique structures
        lambda: ''.join(random.choices(string.ascii_uppercase + string.digits, k=random.randint(5, 10))),  # Variable length Alphanumeric
        lambda: f"{random.choice(string.ascii_uppercase)}{random.randint(10000, 99999)}",  # Single letter followed by 5 digits
        lambda: ",".join([str(random.randint(0, 99)) for _ in range(4)]),  # Comma-separated numbers
    ]

    # Randomly select and call a pattern function
    return random.choice(patterns)()


def generate_random_url():
    # Base URLs for pattern matching
    base_urls = [
        "https://www.youtube.com/watch?v=",
        "http://www.",
        "https://",
        "http://",
        "https://youtu.be/",
        "https://www.linkedin.com/in/",
        "https://www.facebook.com/"
    ]

    # Paths and filenames for pattern matching
    paths = [
        "/wp-content/main/searchregister.htm",
        "/postsabout.html",
        "/category/tags/wp-contentfaq.htm",
        "/tagsmain.html",
        "/explorelogin.htm",
        "/categories/search/tagsmain.html",
        "/main/exploreregister.jsp",
        "/wp-content/appcategory.php",
        "/tag/app/listmain.php",
        "/app/tags/searchregister.php",
        "/tagspost.asp",
        "/main/list/postshomepage.html",
        "/categories/wp-content/tagabout.html",
        "/explore/tagterms.php",
        "/list/appindex.html",
        "/app/tagsauthor.html",
        "/listregister.asp",
        "/categories/appabout.asp",
        "/wp-contenthome.html",
        "/search/categories/wp-contentlogin.htm",
        "/tags/listfaq.html",
        "/wp-content/categorieshomepage.html",
        "/tagmain.html",
        "/appcategory.html",
        "/bloghome.jsp",
        "/categoriesindex.html",
        "/search/wp-content/categorypost.jsp",
        "/posts/wp-content/categoryauthor.php",
        "/tag/searchauthor.html",
        "/listpost.html",
        "/tags/wp-contentpost.html",
        "/blog/search/searchprivacy.php",
        "/list/taghome.php",
        "/wp-content/mainprivacy.htm",
        "/list/explorehomepage.htm",
        "/blog/main/appfaq.jsp",
        "/wp-contentcategory.php",
        "/tags/explore/blogpost.php",
        "/app/categoriesindex.php",
        "/search/tags/tagsearch.php",
        "/posts/searchauthor.php",
        "/mainlogin.htm",
        "/tag/tags/categoriesprivacy.htm",
        "/postsindex.jsp",
        "/wp-content/app/categorieshomepage.jsp",
        "/tagssearch.php",
        "/posts/tagspost.htm",
        "/list/taghomepage.htm",
        "/main/tag/listindex.htm",
        "/blogprivacy.asp",
        "/exploreprivacy.html",
        "/exploreabout.jsp",
        "/wp-content/wp-contentpost.html",
        "/main/posts/tagprivacy.asp",
        "/wp-content/category/bloghome.php",
        "/list/postshomepage.jsp",
        "/posts/blog/searchabout.html",
        "/exploremain.html",
        "/blogsearch.html",
        "/tag/blogindex.asp",
        "/category/wp-contentregister.htm",
        "/categorycategory.jsp",
        "/list/exploreauthor.html",
        "/category/tagprivacy.php",
        "/tag/wp-contentlogin.htm",
        "/categories/search/tagprivacy.htm"
    ]

    # Randomly choose a base URL and a path
    base_url = random.choice(base_urls)
    path = random.choice(paths)

    # Generate a random string for YouTube video IDs or domain names
    if "youtube" in base_url or "youtu.be" in base_url:
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=11))
    else:
        domain = ''.join(random.choices(string.ascii_lowercase, k=random.randint(5, 10)))
        domain_extension = random.choice([".com", ".biz", ".info", ".net", ".org"])
        random_string = domain + domain_extension

    # Combine elements to form the URL
    if "linkedin.com/in/" in base_url or "facebook.com/" in base_url:
        name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(5, 10)))
        random_url = base_url + name
    elif "youtube" in base_url or "youtu.be" in base_url:
        random_url = base_url + random_string
    else:
        random_url = base_url + random_string + path

    return random_url



def generate_fake_pii(pii_class):
    if pii_class == 'EMAIL':
        return fake.email().split("@")[0]+"@"+random.choice(common_email_domains)
    elif pii_class == 'ID_NUM':
        # Simulate an ID number with a random number; adjust format as needed
        return str(generate_random_id())
    elif pii_class == 'NAME_STUDENT':
        return fake.name()
    elif pii_class == 'PHONE_NUM':
        return fake.phone_number().replace("-"," - ").replace("(","( ")
    elif pii_class == 'STREET_ADDRESS':
        return fake.address().replace("\n", "  \n")
    elif pii_class == 'URL_PERSONAL':
        # Generate a simple personal URL
        return generate_random_url()
    elif pii_class == 'USERNAME':
        return fake.user_name()
    else:
        raise



In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")


def tokenize(text):
  # Process the sentence
    doc = nlp(text)

    # Initialize our dictionary with empty lists for tokens, locations, and trailing whitespace flags
    token_details = {"tokens": [], "locations": [], "trailing_whitespace": []}

    # Iterate through the tokens in the document
    for token in doc:
        # Add the token text to the 'tokens' list
        token_details["tokens"].append(token.text)

        # Calculate and add the start and end indices (including trailing space if present)
        start, end = token.idx, token.idx + len(token.text)
        if token.whitespace_:
            end += 1
        token_details["locations"].append((start, end))

        # Add a boolean flag indicating the presence of trailing whitespace
        token_details["trailing_whitespace"].append(bool(token.whitespace_))

    # Printing the structured dictionary
    return token_details


In [ ]:
def find_substring_locations(large_string, substring):
    start = 0
    locations = []
    while True:
        # Find next index of substring, starting search at `start`
        index = large_string.find(substring, start)
        if index == -1:  # If no more occurrences are found, break the loop
            break
        locations.append((index, index+len(substring)))  # Add the found index to the list
        start = index + 1  # Move the start to the next position for the next search
    return locations

def allign_labels(token_details, entities, essay):
    l = len(token_details["tokens"])
    labels = ['O']*l
    full_text = ""
    for i in range(len(token_details["tokens"])):
        if token_details["trailing_whitespace"][i]:
            full_text+=token_details["tokens"][i]+" "
        else:
            full_text+=token_details["tokens"][i]

    for e in entities:
        if entities[e]:
            #print(e)
            if e == 'NAME_STUDENT':
                pieces = entities[e][0].split()
                for j in range(len(pieces)):
                    for i in range(len(token_details["tokens"])):
                        if token_details["tokens"][i] == pieces[j].strip():
                            labels[i] = ['B-', 'I-'][j!=0]+e
            elif e == 'STREET_ADDRESS':
                format_1 = entities[e][0]
                format_2 = entities[e][0].replace("\n", " \n")
                format_3 = entities[e][0].replace("\n", "  \n")
                format_4 = entities[e][0].replace("  \n", "\n")
                locations_1 = find_substring_locations(full_text, format_1)
                locations_2 = find_substring_locations(full_text, format_2)
                locations_3 = find_substring_locations(full_text, format_3)
                locations_4 = find_substring_locations(full_text, format_4)
                locations = set(locations_1 + locations_2 + locations_3+locations_4)

                for location in locations:
                    first=False
                    for i in range(len(token_details["tokens"])):
                        if token_details["locations"][i][1] > location[0] and token_details["locations"][i][0] < location[1]:
                            if not first:
                                labels[i] = 'B-'+e
                                first = True
                            else:
                                labels[i] = 'I-'+e
            else:
                locations = find_substring_locations(full_text, entities[e][0])
                #print(e, locations)
                for location in locations:
                    first=False
                    for i in range(len(token_details["tokens"])):
                        if token_details["locations"][i][1] > location[0] and token_details["locations"][i][0] < location[1]:
                            if not first:
                                labels[i] = 'B-'+e
                                first = True
                            else:
                                labels[i] = 'I-'+e

    return labels


In [ ]:
from openai import OpenAI
api_key = '<api-key>'

client = OpenAI(api_key=api_key)

def generate_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
        ],
        stream=False,
        temperature=0.7,
        max_tokens = 4000
    )
    return response.choices[0].message.content.strip()


In [ ]:
from tqdm import tqdm
import time
documents = train
full_entities_dict = process_documents_to_dict_with_sets(documents)
try:
    with open("./generated_raw_data.json", "r") as fp:
        results = json.load(fp)
except:
    results = []
for c in range(10):
    for entities, _, full_text in tqdm(full_entities_dict):
        piis=""
        generated_entities = {
          "EMAIL":[],
          "ID_NUM":[],
          "USERNAME":[],
          "PHONE_NUM":[],
          "STREET_ADDRESS":[],
          "URL_PERSONAL":[],
          "NAME_STUDENT":[]
        }
        for k in entities.keys():
            generated_entity = generate_fake_pii(k)
            generated_entities[k].append(generated_entity)
            piis+=list(entities[k])[-1]+"({}) => {}".format(k, generated_entity)+",\n"
        prompt = prompt_text.format(piis.strip("\n,"), full_text)
        essay = generate_response(prompt)
        result = {
          "prompt":prompt,
          "essay":essay,
          "entities":generated_entities
        }
        results.append(result)
        with open("./generated_raw_data.json", "w") as fp:
            json.dump(results, fp)
        time.sleep(2)


    final_documents = []
    for result in tqdm(results):
        token_details = tokenize(result["essay"])
        labels = allign_labels(token_details, result["entities"], result["essay"])
        final_documents.append({"tokens": token_details["tokens"], "trailing_whitespace": token_details["trailing_whitespace"], "labels": labels})

    with open("./generated_tokenized_data.json", "w") as fp:
        json.dump(final_documents, fp)